In [32]:
import pandas as pd
import numpy as np
from matplotlib import font_manager, rc

import warnings
warnings.filterwarnings("ignore")
     
time_age = pd.read_csv(r'./data/PatientInfo.csv')
patient_route = pd.read_csv(r'./data/PatientRoute.csv')

joined_df = time_age.merge(patient_route, left_on = 'patient_id', right_on = 'patient_id', how = 'inner')
joined_df = joined_df[joined_df.age.notnull()]
joined_df = joined_df[joined_df.age=='20s']
print(joined_df.type.value_counts())

many_people_place = ['hospital','public_transportation', 'airport', 'church', 'university','school','park']
len(many_people_place)

def ismany(x):
    if x in many_people_place:
        res = True
    else : 
        res = False
    return res

joined_df['is_many'] = joined_df['type'].apply(lambda x: ismany(x))
joined_df = joined_df[['patient_id', 'type', 'is_many']]
joined_df.is_many.value_counts()

etc                            541
hospital                       472
store                          178
restaurant                     167
public_transportation          156
airport                         98
church                          55
cafe                            52
pharmacy                        47
pc_cafe                         43
lodging                         12
academy                         10
university                      10
bar                              8
post_office                      8
gym                              7
bakery                           7
school                           7
bank                             5
karaoke                          5
administrative_area_level_1      3
gas_station                      3
park                             2
beauty_salon                     2
Name: type, dtype: int64


False    1098
True      800
Name: is_many, dtype: int64